In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# import

In [ ]:
import pandas as pd
import os

#정규 표현식을 위한 모듈
import re

#횟수세기
from collections import Counter

from collections import OrderedDict

import numpy as np

# 데이터 불러오기

In [ ]:
origin_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/project/save_review/origin_df1.csv')
print(origin_df)

In [ ]:
origin_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8275 entries, 0 to 8274
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      8275 non-null   int64  
 1   subsr           8275 non-null   int64  
 2   asset_nm        8275 non-null   object 
 3   ct_cl           8275 non-null   object 
 4   genre_of_ct_cl  8275 non-null   object 
 5   use_tms         8275 non-null   int64  
 6   SMRY            8273 non-null   object 
 7   ACTR_DISP       8273 non-null   object 
 8   disp_rtm        8275 non-null   object 
 9   strt_dt         8275 non-null   int64  
 10  month           8275 non-null   int64  
 11  disp_rtm_s      8275 non-null   int64  
 12  asset_nm_new    8275 non-null   object 
 13  date            8275 non-null   object 
 14  day             8275 non-null   object 
 15  hour            8275 non-null   int64  
 16  runningtime     8275 non-null   object 
 17  weather         8275 non-null   o

# 데이터 전처리

**중복된 asset_nm_new 행 제거**

In [ ]:
#origin_df 에서 asset_nm_new가 중복된 값 제거
# 중복된 'asset_nm' 제거
unique_asset = origin_df.drop_duplicates(subset=['asset_nm_new'])

# 결과 출력
print(unique_asset)

**'subsr' ,'asset_nm_new', 'minus_tokens' 빼고 제거**

In [ ]:
columns_to_keep = ['subsr', 'asset_nm_new', 'minus_tokens']
unique_asset = unique_asset.drop(columns=set(unique_asset.columns) - set(columns_to_keep))

print(unique_asset)

         subsr   asset_nm_new  \
0     65941000        그것이알고싶다   
4     61619000  꼬리에꼬리를무는그날이야기   
7     66885000           인간극장   
9     65954000     오은영리포트결혼지옥   
11    67055000          실화탐사대   
...        ...            ...   
8267  60169000            커졌다   
8268  66317000        호랑이뱃속잔치   
8270  62844000         엄마가화났다   
8271  60169000           간질간질   
8274  60169000            놀자!   

                                           minus_tokens  
0     버닝썬,이후,유정,이슈,사건,정리,다음,조사,프로그램,평생,파워,문성근,김상중,제도...  
4     방송,진행,방식,불만,진행자,만담,스킵,정규,편성,방영,이야기,프로,하나,내용,치즈...  
7     시그널,뮤직,역대,다양,사람,세상,기억,에피소드,프로,자체,상징,재미,보장,시민,일...  
9     눈물,사람,느낌,방송,고민,상담,법원,경찰,신고,인간,오은영,박사,저울,부부,소득,...  
11                                                  NaN  
...                                                 ...  
8267                                                NaN  
8268                                                NaN  
8270                                                NaN  
8271                   

**데이터 저장**

In [ ]:
unique_asset.to_csv('/content/drive/MyDrive/Colab Notebooks/project/recommend_keywords/unique_asset.csv', index=False)

# 유저별 중복된 키워드 추출

**데이터 불러오기**

In [ ]:
unique_asset1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/project/recommend_keywords/unique_asset.csv')

In [ ]:
print(unique_asset1)

        subsr   asset_nm_new  \
0    65941000        그것이알고싶다   
1    61619000  꼬리에꼬리를무는그날이야기   
2    66885000           인간극장   
3    65954000     오은영리포트결혼지옥   
4    67055000          실화탐사대   
..        ...            ...   
961  60169000            커졌다   
962  66317000        호랑이뱃속잔치   
963  62844000         엄마가화났다   
964  60169000           간질간질   
965  60169000            놀자!   

                                          minus_tokens  
0    버닝썬,이후,유정,이슈,사건,정리,다음,조사,프로그램,평생,파워,문성근,김상중,제도...  
1    방송,진행,방식,불만,진행자,만담,스킵,정규,편성,방영,이야기,프로,하나,내용,치즈...  
2    시그널,뮤직,역대,다양,사람,세상,기억,에피소드,프로,자체,상징,재미,보장,시민,일...  
3    눈물,사람,느낌,방송,고민,상담,법원,경찰,신고,인간,오은영,박사,저울,부부,소득,...  
4                                                  NaN  
..                                                 ...  
961                                                NaN  
962                                                NaN  
963                                                NaN  
964                                          

## 각 유저마다 가지고 있는 토큰 값을 모두 합치기

In [ ]:
# 새로운 데이터프레임 생성
new_df = pd.DataFrame(columns=['subsr', 'all_tokens'])

In [ ]:
# # 각 행을 순회하면서 토큰 값을 추출하여 새로운 데이터프레임에 추가
# for index, row in unique_asset1.iterrows():
#     subsr = row['subsr']
#     minus_tokens = row['minus_tokens']

#     # NaN 값이 아닌 경우에만 해당 행 추가
#     if isinstance(minus_tokens, list) and any(pd.notna(token) for token in minus_tokens):
#         # 모든 토큰 값을 따옴표로 감싸서 하나의 리스트로 합침
#         all_tokens_flat = [f"'{token}'" for sublist in minus_tokens for token in sublist]

#         # 새로운 데이터프레임에 행 추가
#         new_df = new_df.append({'subsr': subsr, 'all_tokens': all_tokens_flat}, ignore_index=True)

# # 결과 출력
# print(new_df)

        subsr                                         all_tokens
0    65941000  [버, 닝, 썬, ,, 이, 후, ,, 유, 정, ,, 이, 슈, ,, 사, 건, ...
1    61619000  [방, 송, ,, 진, 행, ,, 방, 식, ,, 불, 만, ,, 진, 행, 자, ...
2    66885000  [시, 그, 널, ,, 뮤, 직, ,, 역, 대, ,, 다, 양, ,, 사, 람, ...
3    65954000  [눈, 물, ,, 사, 람, ,, 느, 낌, ,, 방, 송, ,, 고, 민, ,, ...
4    65696000  [안, 현, 모, ,, 감, 정, ,, 패, 널, ,, 편, 안, ,, 엄, 여, ...
..        ...                                                ...
455  67008000  [측, 은, ,, 정, 도, ,, 사, 람, ,, 후, 쿠, 다, ,, 유, 이, ...
456  63086000  [유, 투, 브, ,, 요, 즘, ,, 턱, 스, ,, 수, 술, ,, 동, 물, ...
457  66900000  [조, 금, ,, 아, 이, ,, 뽀, 로, 로, ,, 매, 력, ,, 포, 비, ...
458  65724000  [시, 간, ,, 슬, 랩, 스, 틱, ,, 강, 조, ,, 유, 머, ,, 취, ...
459  60169000                                    [여, 행, ,, 색, 감]

[460 rows x 2 columns]


In [ ]:
# 각 행을 순회하면서 토큰 값을 추출하여 새로운 데이터프레임에 추가
for index, row in unique_asset1.iterrows():
    subsr = row['subsr']
    minus_tokens = row['minus_tokens']

    # NaN 값이면 해당 행 무시
    if pd.notna(minus_tokens):
        # 결측치 또는 실수형을 문자열로 변환 후 ','를 기준으로 분리하여 세트로 변환
        origin_tokens = set(str(minus_tokens).split(','))

        # 새로운 데이터프레임에 행 추가
        new_df = new_df.append({'subsr': subsr, 'all_tokens': list(origin_tokens)}, ignore_index=True)

# 결과 출력
print(new_df)


<ipython-input-60-d058bf93ab5a>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({'subsr': subsr, 'all_tokens': list(origin_tokens)}, ignore_index=True)
<ipython-input-60-d058bf93ab5a>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({'subsr': subsr, 'all_tokens': list(origin_tokens)}, ignore_index=True)
<ipython-input-60-d058bf93ab5a>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({'subsr': subsr, 'all_tokens': list(origin_tokens)}, ignore_index=True)
<ipython-input-60-d058bf93ab5a>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({'subsr'

        subsr                                         all_tokens
0    65941000  [에피소드, 의문, 여성, 정의, 돌멩이, 버닝썬, 사람, 유영철, 분노, 오세훈,...
1    61619000  [옥한흠, 화염, 정반대, 기도, 사람, 과잉, 진행자, 자비, 시작, 희생, 얼마...
2    66885000  [시그널, 에피소드, 자체, 의의, 고정, 프로, 재미, 사람, 프로그램, 역대, ...
3    65954000  [집중, 불행, 시청자, 모자이크, 자체, 가구, 다행, 가치관, 수준, 도덕, 전...
4    65696000  [감정, 니코틴, 살인, 거여동, 연쇄, 치료, 살인마, 아들, 선고, 담배, 행복...
..        ...                                                ...
455  67008000  [칸나, 일본, 의문, 감독, 치의, 탄식, 머리, 디즈니, 사람, 뼛속, 한스, ...
456  63086000  [집중, 새벽, 자체, 수준, 감독, 호러, 유발, 실화, 최악, 사람, 턱스, 창...
457  66900000  [호감, 아이, 매력, 어른, 일지, 동화, 조금, 포비, 마법, 뽀로로, 제외, 사랑]
458  65724000  [라바, 단편, 과거, 시궁창, 케릭, 자랑, 생활, 감동, 한계, 학교, 뽀로로,...
459  60169000                                           [색감, 여행]

[460 rows x 2 columns]


<ipython-input-60-d058bf93ab5a>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({'subsr': subsr, 'all_tokens': list(origin_tokens)}, ignore_index=True)
<ipython-input-60-d058bf93ab5a>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({'subsr': subsr, 'all_tokens': list(origin_tokens)}, ignore_index=True)
<ipython-input-60-d058bf93ab5a>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({'subsr': subsr, 'all_tokens': list(origin_tokens)}, ignore_index=True)
<ipython-input-60-d058bf93ab5a>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({'subsr'

In [ ]:
new_df

,subsr,all_tokens
0,65941000,"[에피소드, 의문, 여성, 정의, 돌멩이, 버닝썬, 사람, 유영철, 분노, 오세훈,..."
1,61619000,"[옥한흠, 화염, 정반대, 기도, 사람, 과잉, 진행자, 자비, 시작, 희생, 얼마..."
2,66885000,"[시그널, 에피소드, 자체, 의의, 고정, 프로, 재미, 사람, 프로그램, 역대, ..."
3,65954000,"[집중, 불행, 시청자, 모자이크, 자체, 가구, 다행, 가치관, 수준, 도덕, 전..."
4,65696000,"[감정, 니코틴, 살인, 거여동, 연쇄, 치료, 살인마, 아들, 선고, 담배, 행복..."
...,...,...
455,67008000,"[칸나, 일본, 의문, 감독, 치의, 탄식, 머리, 디즈니, 사람, 뼛속, 한스, ..."
456,63086000,"[집중, 새벽, 자체, 수준, 감독, 호러, 유발, 실화, 최악, 사람, 턱스, 창..."
457,66900000,"[호감, 아이, 매력, 어른, 일지, 동화, 조금, 포비, 마법, 뽀로로, 제외, 사랑]"
458,65724000,"[라바, 단편, 과거, 시궁창, 케릭, 자랑, 생활, 감동, 한계, 학교, 뽀로로,..."


In [ ]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 460 entries, 0 to 459
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   subsr       460 non-null    object
 1   all_tokens  460 non-null    object
dtypes: object(2)
memory usage: 7.3+ KB


**new_df 저장**

In [ ]:
new_df1.to_csv('/content/drive/MyDrive/Colab Notebooks/project/recommend_keywords/new_df.csv', index=False)

In [ ]:
new_df1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/project/recommend_keywords/new_df.csv')
print(new_df1)

        subsr                                         all_tokens
0    65941000  에피소드, 의문, 여성, 정의, 돌멩이, 버닝썬, 사람, 유영철, 분노, 오세훈, ...
1    61619000  옥한흠, 화염, 정반대, 기도, 사람, 과잉, 진행자, 자비, 시작, 희생, 얼마,...
2    66885000  시그널, 에피소드, 자체, 의의, 고정, 프로, 재미, 사람, 프로그램, 역대, 세...
3    65954000  집중, 불행, 시청자, 모자이크, 자체, 가구, 다행, 가치관, 수준, 도덕, 전문...
4    65696000  감정, 니코틴, 살인, 거여동, 연쇄, 치료, 살인마, 아들, 선고, 담배, 행복,...
..        ...                                                ...
455  67008000  칸나, 일본, 의문, 감독, 치의, 탄식, 머리, 디즈니, 사람, 뼛속, 한스, 콜...
456  63086000  집중, 새벽, 자체, 수준, 감독, 호러, 유발, 실화, 최악, 사람, 턱스, 창의...
457  66900000    호감, 아이, 매력, 어른, 일지, 동화, 조금, 포비, 마법, 뽀로로, 제외, 사랑
458  65724000  라바, 단편, 과거, 시궁창, 케릭, 자랑, 생활, 감동, 한계, 학교, 뽀로로, ...
459  60169000                                             색감, 여행

[460 rows x 2 columns]


In [ ]:
new_df1

,subsr,all_tokens
0,65941000,"[에피소드, 의문, 여성, 정의, 돌멩이, 버닝썬, 사람, 유영철, 분노, 오세훈,..."
1,61619000,"[옥한흠, 화염, 정반대, 기도, 사람, 과잉, 진행자, 자비, 시작, 희생, 얼마..."
2,66885000,"[시그널, 에피소드, 자체, 의의, 고정, 프로, 재미, 사람, 프로그램, 역대, ..."
3,65954000,"[집중, 불행, 시청자, 모자이크, 자체, 가구, 다행, 가치관, 수준, 도덕, 전..."
4,65696000,"[감정, 니코틴, 살인, 거여동, 연쇄, 치료, 살인마, 아들, 선고, 담배, 행복..."
...,...,...
455,67008000,"[칸나, 일본, 의문, 감독, 치의, 탄식, 머리, 디즈니, 사람, 뼛속, 한스, ..."
456,63086000,"[집중, 새벽, 자체, 수준, 감독, 호러, 유발, 실화, 최악, 사람, 턱스, 창..."
457,66900000,"[호감, 아이, 매력, 어른, 일지, 동화, 조금, 포비, 마법, 뽀로로, 제외, 사랑]"
458,65724000,"[라바, 단편, 과거, 시궁창, 케릭, 자랑, 생활, 감동, 한계, 학교, 뽀로로,..."


### new_df 전처리

**' 제거**

In [ ]:
# all_tokens 컬럼에서 작은 따옴표 삭제
new_df1['all_tokens'] = new_df1['all_tokens'].str.replace("'", '')

# 결과 확인
print(new_df1)

        subsr                                         all_tokens
0    65941000  에피소드, 의문, 여성, 정의, 돌멩이, 버닝썬, 사람, 유영철, 분노, 오세훈, ...
1    61619000  옥한흠, 화염, 정반대, 기도, 사람, 과잉, 진행자, 자비, 시작, 희생, 얼마,...
2    66885000  시그널, 에피소드, 자체, 의의, 고정, 프로, 재미, 사람, 프로그램, 역대, 세...
3    65954000  집중, 불행, 시청자, 모자이크, 자체, 가구, 다행, 가치관, 수준, 도덕, 전문...
4    65696000  감정, 니코틴, 살인, 거여동, 연쇄, 치료, 살인마, 아들, 선고, 담배, 행복,...
..        ...                                                ...
455  67008000  칸나, 일본, 의문, 감독, 치의, 탄식, 머리, 디즈니, 사람, 뼛속, 한스, 콜...
456  63086000  집중, 새벽, 자체, 수준, 감독, 호러, 유발, 실화, 최악, 사람, 턱스, 창의...
457  66900000    호감, 아이, 매력, 어른, 일지, 동화, 조금, 포비, 마법, 뽀로로, 제외, 사랑
458  65724000  라바, 단편, 과거, 시궁창, 케릭, 자랑, 생활, 감동, 한계, 학교, 뽀로로, ...
459  60169000                                             색감, 여행

[460 rows x 2 columns]


In [ ]:
new_df1

,subsr,all_tokens
0,65941000,"에피소드, 의문, 여성, 정의, 돌멩이, 버닝썬, 사람, 유영철, 분노, 오세훈, ..."
1,61619000,"옥한흠, 화염, 정반대, 기도, 사람, 과잉, 진행자, 자비, 시작, 희생, 얼마,..."
2,66885000,"시그널, 에피소드, 자체, 의의, 고정, 프로, 재미, 사람, 프로그램, 역대, 세..."
3,65954000,"집중, 불행, 시청자, 모자이크, 자체, 가구, 다행, 가치관, 수준, 도덕, 전문..."
4,65696000,"감정, 니코틴, 살인, 거여동, 연쇄, 치료, 살인마, 아들, 선고, 담배, 행복,..."
...,...,...
455,67008000,"칸나, 일본, 의문, 감독, 치의, 탄식, 머리, 디즈니, 사람, 뼛속, 한스, 콜..."
456,63086000,"집중, 새벽, 자체, 수준, 감독, 호러, 유발, 실화, 최악, 사람, 턱스, 창의..."
457,66900000,"호감, 아이, 매력, 어른, 일지, 동화, 조금, 포비, 마법, 뽀로로, 제외, 사랑"
458,65724000,"라바, 단편, 과거, 시궁창, 케릭, 자랑, 생활, 감동, 한계, 학교, 뽀로로, ..."


**[,] 제거**

In [ ]:
# all_tokens 컬럼에서 [와 ]를 제거
new_df1['all_tokens'] = new_df1['all_tokens'].str.replace('[', '').str.replace(']', '')

# 결과 확인
print(new_df1)

        subsr                                         all_tokens
0    65941000  에피소드, 의문, 여성, 정의, 돌멩이, 버닝썬, 사람, 유영철, 분노, 오세훈, ...
1    61619000  옥한흠, 화염, 정반대, 기도, 사람, 과잉, 진행자, 자비, 시작, 희생, 얼마,...
2    66885000  시그널, 에피소드, 자체, 의의, 고정, 프로, 재미, 사람, 프로그램, 역대, 세...
3    65954000  집중, 불행, 시청자, 모자이크, 자체, 가구, 다행, 가치관, 수준, 도덕, 전문...
4    65696000  감정, 니코틴, 살인, 거여동, 연쇄, 치료, 살인마, 아들, 선고, 담배, 행복,...
..        ...                                                ...
455  67008000  칸나, 일본, 의문, 감독, 치의, 탄식, 머리, 디즈니, 사람, 뼛속, 한스, 콜...
456  63086000  집중, 새벽, 자체, 수준, 감독, 호러, 유발, 실화, 최악, 사람, 턱스, 창의...
457  66900000    호감, 아이, 매력, 어른, 일지, 동화, 조금, 포비, 마법, 뽀로로, 제외, 사랑
458  65724000  라바, 단편, 과거, 시궁창, 케릭, 자랑, 생활, 감동, 한계, 학교, 뽀로로, ...
459  60169000                                             색감, 여행

[460 rows x 2 columns]


<ipython-input-97-44c3a24ceb32>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  new_df1['all_tokens'] = new_df1['all_tokens'].str.replace('[', '').str.replace(']', '')


**띄어쓰기 제거**

In [ ]:
# all_tokens 컬럼에서 띄어쓰기 제거
new_df1['all_tokens'] = new_df1['all_tokens'].str.replace(' ', '')

# 결과 확인
print(new_df1)

        subsr                                         all_tokens
0    65941000  에피소드,의문,여성,정의,돌멩이,버닝썬,사람,유영철,분노,오세훈,진행자,얼마,덕후,...
1    61619000  옥한흠,화염,정반대,기도,사람,과잉,진행자,자비,시작,희생,얼마,트롤리,한인,대성,...
2    66885000  시그널,에피소드,자체,의의,고정,프로,재미,사람,프로그램,역대,세상,뮤직,장수,대중...
3    65954000  집중,불행,시청자,모자이크,자체,가구,다행,가치관,수준,도덕,전문가,진정,개념,홧팅...
4    65696000  감정,니코틴,살인,거여동,연쇄,치료,살인마,아들,선고,담배,행복,여고,사회,장애,패...
..        ...                                                ...
455  67008000  칸나,일본,의문,감독,치의,탄식,머리,디즈니,사람,뼛속,한스,콜라,불가,작품,주의,...
456  63086000  집중,새벽,자체,수준,감독,호러,유발,실화,최악,사람,턱스,창의,장편,판매량,프리뷰...
457  66900000               호감,아이,매력,어른,일지,동화,조금,포비,마법,뽀로로,제외,사랑
458  65724000  라바,단편,과거,시궁창,케릭,자랑,생활,감동,한계,학교,뽀로로,재미,생각,경기도,친...
459  60169000                                              색감,여행

[460 rows x 2 columns]


In [ ]:
new_df1

,subsr,all_tokens
0,65941000,"에피소드,의문,여성,정의,돌멩이,버닝썬,사람,유영철,분노,오세훈,진행자,얼마,덕후,..."
1,61619000,"옥한흠,화염,정반대,기도,사람,과잉,진행자,자비,시작,희생,얼마,트롤리,한인,대성,..."
2,66885000,"시그널,에피소드,자체,의의,고정,프로,재미,사람,프로그램,역대,세상,뮤직,장수,대중..."
3,65954000,"집중,불행,시청자,모자이크,자체,가구,다행,가치관,수준,도덕,전문가,진정,개념,홧팅..."
4,65696000,"감정,니코틴,살인,거여동,연쇄,치료,살인마,아들,선고,담배,행복,여고,사회,장애,패..."
...,...,...
455,67008000,"칸나,일본,의문,감독,치의,탄식,머리,디즈니,사람,뼛속,한스,콜라,불가,작품,주의,..."
456,63086000,"집중,새벽,자체,수준,감독,호러,유발,실화,최악,사람,턱스,창의,장편,판매량,프리뷰..."
457,66900000,"호감,아이,매력,어른,일지,동화,조금,포비,마법,뽀로로,제외,사랑"
458,65724000,"라바,단편,과거,시궁창,케릭,자랑,생활,감동,한계,학교,뽀로로,재미,생각,경기도,친..."


# new_df1와

In [ ]:
#데이터 불러오기
new_df1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/project/recommend_keywords/new_df.csv')
print(new_df1)

        subsr                                         all_tokens
0    65941000  에피소드,의문,여성,정의,돌멩이,버닝썬,사람,유영철,분노,오세훈,진행자,얼마,덕후,...
1    61619000  옥한흠,화염,정반대,기도,사람,과잉,진행자,자비,시작,희생,얼마,트롤리,한인,대성,...
2    66885000  시그널,에피소드,자체,의의,고정,프로,재미,사람,프로그램,역대,세상,뮤직,장수,대중...
3    65954000  집중,불행,시청자,모자이크,자체,가구,다행,가치관,수준,도덕,전문가,진정,개념,홧팅...
4    65696000  감정,니코틴,살인,거여동,연쇄,치료,살인마,아들,선고,담배,행복,여고,사회,장애,패...
..        ...                                                ...
455  67008000  칸나,일본,의문,감독,치의,탄식,머리,디즈니,사람,뼛속,한스,콜라,불가,작품,주의,...
456  63086000  집중,새벽,자체,수준,감독,호러,유발,실화,최악,사람,턱스,창의,장편,판매량,프리뷰...
457  66900000               호감,아이,매력,어른,일지,동화,조금,포비,마법,뽀로로,제외,사랑
458  65724000  라바,단편,과거,시궁창,케릭,자랑,생활,감동,한계,학교,뽀로로,재미,생각,경기도,친...
459  60169000                                              색감,여행

[460 rows x 2 columns]


In [ ]:
# 결과를 저장할 빈 리스트 생성
result_rows = []

# 각 행마다 비교 수행
for index1, row1 in unique_asset1.iterrows():
    for index2, row2 in new_df1.iterrows():
        # nan 값 체크 후 공통 토큰 찾기
        common_tokens = set(row1['minus_tokens']) & set(row2['all_tokens']) if not (pd.isna(row1['minus_tokens']) or pd.isna(row2['all_tokens'])) else set()

        # 결측치 또는 실수형을 문자열로 변환 후 ','를 기준으로 분리하여 세트로 변환
        origin_tokens = set(str(row1['minus_tokens']).split(',')) if not pd.isna(row1['minus_tokens']) else set()

        if common_tokens:
            result_rows.append({'subsr': row2['subsr'],
                                'asset_nm_new': row1['asset_nm_new'],
                                'origin_tokens': list(origin_tokens)})

# 결과를 데이터프레임으로 변환
result_df = pd.DataFrame(result_rows)

# 결과 출력
print(result_df)

           subsr asset_nm_new  \
0       65941000      그것이알고싶다   
1       61619000      그것이알고싶다   
2       66885000      그것이알고싶다   
3       65954000      그것이알고싶다   
4       65696000      그것이알고싶다   
...          ...          ...   
209573  67008000         간질간질   
209574  63086000         간질간질   
209575  66900000         간질간질   
209576  65724000         간질간질   
209577  60169000         간질간질   

                                            origin_tokens  
0       [에피소드, 의문, 여성, 정의, 돌멩이, 버닝썬, 사람, 유영철, 분노, 오세훈,...  
1       [에피소드, 의문, 여성, 정의, 돌멩이, 버닝썬, 사람, 유영철, 분노, 오세훈,...  
2       [에피소드, 의문, 여성, 정의, 돌멩이, 버닝썬, 사람, 유영철, 분노, 오세훈,...  
3       [에피소드, 의문, 여성, 정의, 돌멩이, 버닝썬, 사람, 유영철, 분노, 오세훈,...  
4       [에피소드, 의문, 여성, 정의, 돌멩이, 버닝썬, 사람, 유영철, 분노, 오세훈,...  
...                                                   ...  
209573                                           [색감, 여행]  
209574                                           [색감, 여행]  
209575                                           [색감, 여행]  
209

In [ ]:
# count 컬럼 추가 및 토큰 개수 세기
result_df['count'] = result_df['origin_tokens'].apply(lambda x: len(x))

**result_df 저장**

In [ ]:
result_df.to_csv('/content/drive/MyDrive/Colab Notebooks/project/recommend_keywords/result.csv', index=False)

In [ ]:
result_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/project/recommend_keywords/result.csv')
print(result_df)

           subsr   asset_nm_new  \
0       59930000       응답하라1988   
1       59930000           열화영웅   
2       59930000          킬링로맨스   
3       59930000     황금어장-라디오스타   
4       59930000     삼생초월로맨스-우룡   
...          ...            ...   
209573  67106000         인디아나존스   
209574  67106000           상류사회   
209575  67106000  분노의질주:라이드오어다이   
209576  67106000             레드   
209577  67106000          언더파이어   

                                            origin_tokens  count  
0       ['처절', '청춘', '시청자', '자체', '대비', '레파토리', '시대', ...    287  
1       ['에피소드', '다행', '소방관', '인타', '존경', '사람', '중대', ...    177  
2       ['오수', '속칭', '사람', '광고', '휙확훅', '시작', '얼마', '모...    431  
3       ['종신형', '꿀잼', '기본', '사람', '예능', '노잼', '유일', '이...     84  
4       ['머리', '사람', '노잼', '시작', '서사', '정화', '이상', '이해...     88  
...                                                   ...    ...  
209573  ['대작', '탄식', '기도', '사람', '무언', '던전', '향연', '성전...    532  
209574  ['비판', '중류', '반영', '사람', '차이점', '예술

In [ ]:
result_df = result_df.sort_values(by='subsr').reset_index(drop=True)
print(result_df)

           subsr asset_nm_new  \
0       59930000     응답하라1988   
1       59930000        비닐하우스   
2       59930000  늑대와일곱마리아기염소   
3       59930000      유퀴즈온더블럭   
4       59930000           청락   
...          ...          ...   
209573  67106000           루키   
209574  67106000         블레이즈   
209575  67106000     신발벗고돌싱포맨   
209576  67106000        뭉쳐야뜬다   
209577  67106000        언더파이어   

                                            origin_tokens  count  
0       ['처절', '청춘', '시청자', '자체', '대비', '레파토리', '시대', ...    287  
1       ['처절', '집중', '불행', '수준', '다행', '하늘', '주변', '불안...    165  
2                  ['은근', '우리', '오누이', '나라', '연상', '이야기']      6  
3       ['행운', '주변', '감동', '퀴즈', '프로', '대화', '다음', '성실...     69  
4       ['시작', '시퀀스', '배우', '가짜', '액션', '테이크', '진행', '...     12  
...                                                   ...    ...  
209573  ['물리', '집중', '무시', '자체', '수준', '그림', '시놉시스', '...    260  
209574  ['보호', '직감', '작별', '사람', '소름', '향연', '생략', '아이...    363  


**count 컬럼에 데이터 유형 확인하기**

In [ ]:
data_type = result_df['count'].dtype
print(f"count 컬럼의 데이터 유형: {data_type}")

count 컬럼의 데이터 유형: int64


In [ ]:
data_type = result_df['subsr'].dtype
print(f"count 컬럼의 데이터 유형: {data_type}")

count 컬럼의 데이터 유형: int64


In [ ]:
# 각 subsr 값에 대해 count 값을 기준으로 내림차순으로 정렬하고 상위 15개의 행 선택
top3_df = result_df.sort_values(by='count', ascending=False).groupby('subsr').head(15)

print(top3_df)

           subsr asset_nm_new  \
46339   60939000            놉   
3516    60067000            놉   
141204  65696000            놉   
141195  65696000            놉   
141140  65696000            놉   
...          ...          ...   
55310   61683000     신과함께-인과연   
55253   61683000         가버나움   
55264   61683000           레드   
55328   61683000     신과함께-죄와벌   
55293   61683000    어쩌다마주친,그대   

                                            origin_tokens  count  
46339   ['플랫폼', '카트', '비판', '시청자', '시놉시스', '국내', '호러',...   1180  
3516    ['플랫폼', '카트', '비판', '시청자', '시놉시스', '국내', '호러',...   1180  
141204  ['플랫폼', '카트', '비판', '시청자', '시놉시스', '국내', '호러',...   1180  
141195  ['플랫폼', '카트', '비판', '시청자', '시놉시스', '국내', '호러',...   1180  
141140  ['플랫폼', '카트', '비판', '시청자', '시놉시스', '국내', '호러',...   1180  
...                                                   ...    ...  
55310   ['아우', '탄식', '난립', '사람', '조선', '광고', '독보', '시작...    503  
55253   ['처절', '에피소드', '가공품', '비판', '보호', '터리', '고난', ...    499  


In [ ]:
# subsr 기준으로 먼저 정렬하고, 그 후에 각 subsr 그룹 내에서 count 값을 기준으로 내림차순으로 정렬
sorted_df = top3_df.sort_values(by=['subsr', 'count'], ascending=[True, False])

print(sorted_df)

           subsr   asset_nm_new  \
895     59930000              놉   
906     59930000              놉   
903     59930000              놉   
988     59930000          배드럭뱅잉   
892     59930000          배드럭뱅잉   
...          ...            ...   
209573  67106000         인디아나존스   
209141  67106000  인디아나존스:운명의다이얼   
209548  67106000          은밀한비밀   
209539  67106000  에브리씽에브리웨어올앳원스   
209546  67106000         킹오브몬스터   

                                            origin_tokens  count  
895     ['플랫폼', '카트', '비판', '시청자', '시놉시스', '국내', '호러',...   1180  
906     ['플랫폼', '카트', '비판', '시청자', '시놉시스', '국내', '호러',...   1180  
903     ['플랫폼', '카트', '비판', '시청자', '시놉시스', '국내', '호러',...   1180  
988     ['문구', '비판', '태블릿', '길거리', '보편', '에너지', '르포', ...    927  
892     ['문구', '비판', '태블릿', '길거리', '보편', '에너지', '르포', ...    927  
...                                                   ...    ...  
209573  ['대작', '탄식', '기도', '사람', '무언', '던전', '향연', '성전...    532  
209141  ['대작', '탄식', '기도', '사람', '무언', '던전'

In [ ]:
sorted_df.to_csv('/content/drive/MyDrive/Colab Notebooks/project/recommend_keywords/recommend.csv', index=False)
print(sorted_df)

           subsr   asset_nm_new  \
895     59930000              놉   
906     59930000              놉   
903     59930000              놉   
988     59930000          배드럭뱅잉   
892     59930000          배드럭뱅잉   
...          ...            ...   
209573  67106000         인디아나존스   
209141  67106000  인디아나존스:운명의다이얼   
209548  67106000          은밀한비밀   
209539  67106000  에브리씽에브리웨어올앳원스   
209546  67106000         킹오브몬스터   

                                            origin_tokens  count  
895     ['플랫폼', '카트', '비판', '시청자', '시놉시스', '국내', '호러',...   1180  
906     ['플랫폼', '카트', '비판', '시청자', '시놉시스', '국내', '호러',...   1180  
903     ['플랫폼', '카트', '비판', '시청자', '시놉시스', '국내', '호러',...   1180  
988     ['문구', '비판', '태블릿', '길거리', '보편', '에너지', '르포', ...    927  
892     ['문구', '비판', '태블릿', '길거리', '보편', '에너지', '르포', ...    927  
...                                                   ...    ...  
209573  ['대작', '탄식', '기도', '사람', '무언', '던전', '향연', '성전...    532  
209141  ['대작', '탄식', '기도', '사람', '무언', '던전'

In [ ]:
sorted_df = sorted_df.drop_duplicates(subset=['subsr', 'asset_nm_new'], keep='first')
print(sorted_df)

           subsr   asset_nm_new  \
895     59930000              놉   
988     59930000          배드럭뱅잉   
415     59930000           경경일상   
1309    59930000             우상   
1024    59930000            존윅4   
...          ...            ...   
209573  67106000         인디아나존스   
209141  67106000  인디아나존스:운명의다이얼   
209548  67106000          은밀한비밀   
209539  67106000  에브리씽에브리웨어올앳원스   
209546  67106000         킹오브몬스터   

                                            origin_tokens  count  
895     ['플랫폼', '카트', '비판', '시청자', '시놉시스', '국내', '호러',...   1180  
988     ['문구', '비판', '태블릿', '길거리', '보편', '에너지', '르포', ...    927  
415     ['경정', '여관', '순이', '식당', '곤궁', '통쾌', '손실', '옥패...    722  
1309    ['몰입도', '직감', '정반대', '고난', '사람', '반영', '곤욕', '...    666  
1024    ['처절', '문구', '길거리', '종합', '개념', '일출', '고난', '맹...    636  
...                                                   ...    ...  
209573  ['대작', '탄식', '기도', '사람', '무언', '던전', '향연', '성전...    532  
209141  ['대작', '탄식', '기도', '사람', '무언', '던전'

In [ ]:
sorted_df.reset_index(drop=True)
print(sorted_df)

           subsr   asset_nm_new  \
895     59930000              놉   
988     59930000          배드럭뱅잉   
415     59930000           경경일상   
1309    59930000             우상   
1024    59930000            존윅4   
...          ...            ...   
209573  67106000         인디아나존스   
209141  67106000  인디아나존스:운명의다이얼   
209548  67106000          은밀한비밀   
209539  67106000  에브리씽에브리웨어올앳원스   
209546  67106000         킹오브몬스터   

                                            origin_tokens  count  
895     ['플랫폼', '카트', '비판', '시청자', '시놉시스', '국내', '호러',...   1180  
988     ['문구', '비판', '태블릿', '길거리', '보편', '에너지', '르포', ...    927  
415     ['경정', '여관', '순이', '식당', '곤궁', '통쾌', '손실', '옥패...    722  
1309    ['몰입도', '직감', '정반대', '고난', '사람', '반영', '곤욕', '...    666  
1024    ['처절', '문구', '길거리', '종합', '개념', '일출', '고난', '맹...    636  
...                                                   ...    ...  
209573  ['대작', '탄식', '기도', '사람', '무언', '던전', '향연', '성전...    532  
209141  ['대작', '탄식', '기도', '사람', '무언', '던전'

In [ ]:
sorted_df.to_csv('/content/drive/MyDrive/Colab Notebooks/project/recommend_keywords/recommend.csv', index=False)

In [ ]:
# subsr과 asset_nm_new 값이 같은 행 중복 제거
result_df = result_df.drop_duplicates(subset=['subsr', 'asset_nm_new'], keep='first')

In [ ]:
result_df = result_df.sort_values(by=['subsr', 'count'], ascending=[True, False])
print(result_df)

           subsr asset_nm_new  \
895     59930000            놉   
706     59930000        배드럭뱅잉   
415     59930000         경경일상   
1095    59930000           우상   
1024    59930000          존윅4   
...          ...          ...   
209516  67106000     뱀파이어소녀달자   
209528  67106000        하늘의인연   
209190  67106000          봉신방   
209490  67106000          천기검   
209494  67106000        미스터로또   

                                            origin_tokens  count  
895     ['플랫폼', '카트', '비판', '시청자', '시놉시스', '국내', '호러',...   1180  
706     ['문구', '비판', '태블릿', '길거리', '보편', '에너지', '르포', ...    927  
415     ['경정', '여관', '순이', '식당', '곤궁', '통쾌', '손실', '옥패...    722  
1095    ['몰입도', '직감', '정반대', '고난', '사람', '반영', '곤욕', '...    666  
1024    ['처절', '문구', '길거리', '종합', '개념', '일출', '고난', '맹...    636  
...                                                   ...    ...  
209516                                       ['사랑', '달자']      2  
209528                                      ['실화', '연기력']      2  


In [ ]:
# '나' 또는 '라'가 들어간 행 제거
result_df = result_df[~result_df['origin_tokens'].apply(lambda x: any(token in x for token in ['처음', '시작', '감독', '사람']))]

# 인덱스 재설정
result_df = result_df.reset_index(drop=True)

# 결과 출력
print(result_df)

          subsr    asset_nm_new  \
0      59930000  장화신은고양이:끝내주는모험   
1      59930000         장화신은고양이   
2      59930000           메가로돈2   
3      59930000         법대로사랑하라   
4      59930000   소방서옆경찰서그리고국과수   
...         ...             ...   
18936  67106000        뱀파이어소녀달자   
18937  67106000           하늘의인연   
18938  67106000             봉신방   
18939  67106000             천기검   
18940  67106000           미스터로또   

                                           origin_tokens  count  
0      ['자체', '차차차', '야옹', '출근', '감칠맛', '기반', '좀도둑', ...    171  
1      ['자체', '차차차', '야옹', '출근', '감칠맛', '기반', '좀도둑', ...    171  
2      ['집중', '상치', '다행', '중화사상', '게임', '진출', '승부', '...    169  
3      ['발암', '집중', '에피소드', '불행', '정의', '고난', '채널', '...    142  
4      ['에피소드', '시청자', '재밋', '일본', '소방관', '디테', '희생',...    127  
...                                                  ...    ...  
18936                                       ['사랑', '달자']      2  
18937                                      ['실화', '

In [ ]:
result_df.to_csv('/content/drive/MyDrive/Colab Notebooks/project/recommend_keywords/test_result.csv', index=False)

In [ ]:
result_df = result_df.groupby('subsr').apply(lambda group: group.nlargest(15, 'count')).reset_index(drop=True)
print(result_df)

         subsr   asset_nm_new  \
0     59930000              놉   
1     59930000          배드럭뱅잉   
2     59930000           경경일상   
3     59930000             우상   
4     59930000            존윅4   
...        ...            ...   
2185  67106000         인디아나존스   
2186  67106000  인디아나존스:운명의다이얼   
2187  67106000          은밀한비밀   
2188  67106000  에브리씽에브리웨어올앳원스   
2189  67106000         킹오브몬스터   

                                          origin_tokens  count  
0     ['플랫폼', '카트', '비판', '시청자', '시놉시스', '국내', '호러',...   1180  
1     ['문구', '비판', '태블릿', '길거리', '보편', '에너지', '르포', ...    927  
2     ['경정', '여관', '순이', '식당', '곤궁', '통쾌', '손실', '옥패...    722  
3     ['몰입도', '직감', '정반대', '고난', '사람', '반영', '곤욕', '...    666  
4     ['처절', '문구', '길거리', '종합', '개념', '일출', '고난', '맹...    636  
...                                                 ...    ...  
2185  ['대작', '탄식', '기도', '사람', '무언', '던전', '향연', '성전...    532  
2186  ['대작', '탄식', '기도', '사람', '무언', '던전', '향연', '성전...    527  
2187  ['꺄르륵', '만고'

In [ ]:
result_df.to_csv('/content/drive/MyDrive/Colab Notebooks/project/recommend_keywords/test_result1.csv', index=False)

# 성능평가

**데이터 불러오기**

In [ ]:
#10월데이터 불러오기
vod_10 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/project/test/vod_10.csv')
print(vod_10)

         subsr                                    asset_nm     ct_cl  \
0     67055000                 꼬리에꼬리를무는그날이야기 55회(22/12/01)  TV 시사/교양   
1     65443000                 꼬리에꼬리를무는그날이야기 55회(22/12/01)  TV 시사/교양   
2     65443000                 꼬리에꼬리를무는그날이야기 55회(22/12/01)  TV 시사/교양   
3     65443000                 꼬리에꼬리를무는그날이야기 55회(22/12/01)  TV 시사/교양   
4     65443000                 꼬리에꼬리를무는그날이야기 55회(22/12/01)  TV 시사/교양   
...        ...                                         ...       ...   
3487  61931000                        엉뚱발랄 콩순이와 친구들 4기 01회        키즈   
3488  63396000  Who stole the Cookies from the Cookie Jar?        키즈   
3489  66900000                            핑크퐁 호기의 우주동요 01회        키즈   
3490  65198000                                    더 놀다 잘래요        키즈   
3491  65198000                                        달샤베트        키즈   

     genre_of_ct_cl  use_tms  \
0                기타     4500   
1                기타     4494   
2                기타     2121   
3      

In [ ]:
#vod_10 = 10월 데이터, 중복있는 행 제거
vod_10 = vod_10.drop_duplicates(subset=['subsr', 'asset_nm_new'], keep='first')
print(vod_10)

         subsr                                    asset_nm     ct_cl  \
0     67055000                 꼬리에꼬리를무는그날이야기 55회(22/12/01)  TV 시사/교양   
1     65443000                 꼬리에꼬리를무는그날이야기 55회(22/12/01)  TV 시사/교양   
7     61689000                 (HD)그것이알고싶다 1369회(23/09/16)  TV 시사/교양   
8     60984000                 (HD)그것이알고싶다 1369회(23/09/16)  TV 시사/교양   
9     61141000                 (HD)그것이알고싶다 1369회(23/09/16)  TV 시사/교양   
...        ...                                         ...       ...   
3487  61931000                        엉뚱발랄 콩순이와 친구들 4기 01회        키즈   
3488  63396000  Who stole the Cookies from the Cookie Jar?        키즈   
3489  66900000                            핑크퐁 호기의 우주동요 01회        키즈   
3490  65198000                                    더 놀다 잘래요        키즈   
3491  65198000                                        달샤베트        키즈   

     genre_of_ct_cl  use_tms  \
0                기타     4500   
1                기타     4494   
7                기타     4570   
8      

In [ ]:
final = result_df
print(final)

         subsr   asset_nm_new  \
0     59930000              놉   
1     59930000          배드럭뱅잉   
2     59930000           경경일상   
3     59930000             우상   
4     59930000            존윅4   
...        ...            ...   
2185  67106000         인디아나존스   
2186  67106000  인디아나존스:운명의다이얼   
2187  67106000          은밀한비밀   
2188  67106000  에브리씽에브리웨어올앳원스   
2189  67106000         킹오브몬스터   

                                          origin_tokens  count  
0     ['플랫폼', '카트', '비판', '시청자', '시놉시스', '국내', '호러',...   1180  
1     ['문구', '비판', '태블릿', '길거리', '보편', '에너지', '르포', ...    927  
2     ['경정', '여관', '순이', '식당', '곤궁', '통쾌', '손실', '옥패...    722  
3     ['몰입도', '직감', '정반대', '고난', '사람', '반영', '곤욕', '...    666  
4     ['처절', '문구', '길거리', '종합', '개념', '일출', '고난', '맹...    636  
...                                                 ...    ...  
2185  ['대작', '탄식', '기도', '사람', '무언', '던전', '향연', '성전...    532  
2186  ['대작', '탄식', '기도', '사람', '무언', '던전', '향연', '성전...    527  
2187  ['꺄르륵', '만고'

In [ ]:
final = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/project/recommend_keywords/recommend.csv')
#final.info()
print(final)

         subsr   asset_nm_new  \
0     59930000              놉   
1     59930000          배드럭뱅잉   
2     59930000           경경일상   
3     59930000             우상   
4     59930000            존윅4   
...        ...            ...   
1435  67106000         인디아나존스   
1436  67106000  인디아나존스:운명의다이얼   
1437  67106000          은밀한비밀   
1438  67106000  에브리씽에브리웨어올앳원스   
1439  67106000         킹오브몬스터   

                                          origin_tokens  count  
0     ['플랫폼', '카트', '비판', '시청자', '시놉시스', '국내', '호러',...   1180  
1     ['문구', '비판', '태블릿', '길거리', '보편', '에너지', '르포', ...    927  
2     ['경정', '여관', '순이', '식당', '곤궁', '통쾌', '손실', '옥패...    722  
3     ['몰입도', '직감', '정반대', '고난', '사람', '반영', '곤욕', '...    666  
4     ['처절', '문구', '길거리', '종합', '개념', '일출', '고난', '맹...    636  
...                                                 ...    ...  
1435  ['대작', '탄식', '기도', '사람', '무언', '던전', '향연', '성전...    532  
1436  ['대작', '탄식', '기도', '사람', '무언', '던전', '향연', '성전...    527  
1437  ['꺄르륵', '만고'

In [ ]:
columns=['subsr','asset_nm_new']
final = pd.DataFrame(final, columns=columns)
print(final)

         subsr   asset_nm_new
0     59930000              놉
1     59930000          배드럭뱅잉
2     59930000           경경일상
3     59930000             우상
4     59930000            존윅4
...        ...            ...
2185  67106000         인디아나존스
2186  67106000  인디아나존스:운명의다이얼
2187  67106000          은밀한비밀
2188  67106000  에브리씽에브리웨어올앳원스
2189  67106000         킹오브몬스터

[2190 rows x 2 columns]


In [ ]:
vod_10['total']=1

In [ ]:
final['one'] = 1

In [ ]:
merge_test = pd.merge(vod_10,final, how='left',on = ['subsr','asset_nm_new'])

In [ ]:
merge_test.fillna(0, inplace=True)

In [ ]:
co_recall_test = merge_test['one'].sum() / merge_test['total'].sum()

In [ ]:
co_recall_test

0.0028116213683223993